---

### Student name:Temuulen Ganbold csc427 ps3

---

## DFA Implementation


The MachineModel instantiates a DFA around a machine description. Its compute methond takes a string and returns whether that string is in (True) or not in (False) the language recognized by the DFA. The TestMachine class takes a machine description and a test vector and confirms or not if the test is passed.



In [1]:
"""
The verbose switch:
    Set this true or false, to run code verbosely
"""

verbose = False

In [2]:
class MachineModel:
    """
    A machine description is a dictionary with,
        'states': a list of states.
        'alphabet': a list of letters (strings of length one)
        'transitions': a dictionary with keys tuples (a state,a letter) to a state
        'start': a state (the start state)
        'accept': a list of states (the accepting states)
        
    The states are any hashable, and we use:
    - strings for simple DFA's, 
    - tuples for product DFA's, 
    - and, in next week's problem set, frozensets for determinizing an NFA
        
    """
    
    def __init__(self,machine_description):
        self.states = machine_description['states']
        self.alphabet = machine_description['alphabet']
        self.transitions = machine_description['transitions']
        self.start_state = machine_description['start'] 
        self.accept_states = machine_description['accept']
        self.current_state = self.start_state 

    def do_transition(self,letter):
        self.current_state = self.transitions[(self.current_state,letter)]
    
    def compute(self,word):
        self.current_state = self.start_state
        if verbose : print(self.current_state)
        for w in word:
            self.do_transition(w)
            if verbose : print(w,self.current_state)
        return self.current_state in self.accept_states

    def describe(self,name=""):
        print("Machine Description:",name)
        print("\tstates:",len(self.states))
        for s in self.states:
            print("\t\t",s)
        print("\ttransitions:",len(self.transitions))
        for t,v in self.transitions.items():
            print(f"\t\t{t}  ->  {v}")
        print("\taccept states:",len(self.accept_states))
        for a in self.accept_states:
            print("\t\t",a)
        print()


def test_machine(dfa_description,test_cases,name=""):
    
    print('running tests ...')
    dfa = MachineModel(dfa_description)
    if verbose: dfa.describe(name)
    for (t,r) in (test_cases):
        if dfa.compute(t) != r:
            print(r,'\t|'+t+'|','\tWRONG, ABORT')
            return False
        print(r,'\t|'+t+'|','\tOK')
    return True
  

## The Intersection Code

The objective is to show that the intersection of two regular languages is regular. A DFA is easily complemented, so it is true that the intersection is regular by De Morgan's law, A AND B = NOT(NOT A OR NOT B). However the OR operation takes the two complemented DFA's and makes an NFA, which then has to be converted back to DFA's for the final complement. This can exponentiate the number of states.

In this exercise we build a intersection machine directly. It is in the sprit of the NFA to DFA construction, in that we will simulate the computation of the two DFAs in parallel. We create *product machine* that has states form as pars, $(s,t)$, were $s$ is a state from the first machine, and $t$ is a state from the second machine. To be in state $(s,t)$ means that on the input used up to that moment in the computation, the first machine would be in state $s$ and the second in state $t$. 

There are several possibilities for the accepting states of a product machine, but to calculate the intersection of the two languages, a state $(s,t)$ is accepting in the product machine, if $s$ is accepting in the first machine and $t$ is accepting in the second mahince.

### Mathematical definition.

Given two DFAs,

$$
M_i = \langle \, Q_i, \Sigma, \delta_i, q_i, A_i \rangle,\quad\mbox{ for } i = 1,2, 
$$

the product machine $M$ computing the intersection language 

$$
\cal{L}(M) = \cal{L}(M_1)\cap\cal{L}(M_2),
$$

is given by the DFA,

$$
M = \langle\, \, Q_1 \times Q_2,\,  \Sigma,\, \delta,\, (q_1,q_2),\, A_1 \times A_2 \,\,\rangle,
$$

where

$$
\begin{array}{crcl}
\delta : & (Q_1 \times Q_2) \times \Sigma & \longrightarrow &  Q_1 \times Q_2 \\
 & ( (s, t), \,a) & \mapsto & ( \,\, \delta_1(s,a),\, \delta_2(t,a)\,\,)
\end{array}
$$



In [3]:

class IntersectionDFA:
    
    """
    given two DFA descriptions, create a DFA description that accepts the 
    intersection of the languages of the given DFA
    
    alphabets must be the same
    
    the two DFA's presented are in our standard form; the output DFA makes
    the change that states are 2-tuples of states. since tupes are immuatable,
    as are strings, this change does not affect any coding.
    
    output:
        'states':list(tuple(string,string))
        'alphabet':list(character)
        'transitions':dict(tuple(tuple(string,string),character):tuple(string,string))
        'start':tuple(string,string)
        'accept':list(tuple(string,string))
    
    input:
        'states':list(string)
        'alphabet':list(character)
        'transtions':dict(tuple(string,string):string)
        'start':string
        'accept':list(string)
    
    """
    
    def __init__(self, dfa1, dfa2):
        self.dfa1 = dfa1
        self.dfa2 = dfa2
        self.states = []
        self.alphabet = []
        self.transitions = {}
        self.start = None # an empty 2-tuple should go here, but those do not exist
        self.accept = []
        
        # the alphabets must be the same
        assert set(dfa1['alphabet']) == set(dfa2['alphabet'])

    def cartesian_product(self,l1,l2):
        """
        given l1, l2 each a list(string), return the cartesian product list(tuple(string,string))
        of all pairs  of strings, the first from the first list, the second from the second list
        """
        
       # pass  # write code
        product_list = []
        for i in l1:
            for j in l2:
                product_list.append((i,j))
        return product_list  # replace None as well
    
    def do_transitions(self):
        """
        create transitions of the Intersection DFA.
        - go through all state,letter pairs of the Intersection DFA
        - for each pair, refer to the transitions in dfa1 and dfa to 
          find the resulting state in the Intersection DFA
        """
        for i,state in (self.dfa1['transitions']).items():
            for j,state2 in (self.dfa2['transitions']).items():
                if i[1] == j[1]:
                    self.transitions[((i[0],j[0]),i[1])] = (state, state2)
        
        pass # write code 
        
    def construct(self):
        """
        construct the Intersection DFA.
        - what alphabet does it have
        - what start state does it have
        - what states does it have
        - what accept states does it have
        - what transitions does it have
        """
        self.states = self.cartesian_product(self.dfa1['states'],self.dfa2['states']) 
        self.alphabet = self.dfa2['alphabet']
        self.do_transitions()
        self.start = (self.dfa1['start'], self.dfa2['start'])
        self.accept = self.cartesian_product(self.dfa1['accept'],self.dfa2['accept'])
        
        
        pass  # write code
    
        return {
            'states':self.states,
            'alphabet':self.alphabet,
            'transitions':self.transitions,
            'start':self.start,
            'accept':self.accept
        }



### Simple test cases

In [4]:
# a sample machine

X = {
    'states':['X1','X2','X3','R'],
    'alphabet':['a','b'],
    'transitions':{
        ('X1','b'):'X1',('X1','a'):'X2',
        ('X2','b'):'R',('X2','a'):'X3',
        ('X3','b'):'X3',('X3','a'):'R',
        ('R','a'):'R',('R','b'):'R'
    },
    'start':'X1',
    'accept':['X3']
}                                                   
                                            
Y = {
    'states':['Y1','Y2','Y3','R'],
    'alphabet':['a','b'],
    'transitions':{
        ('Y1','a'):'Y1',('Y1','b'):'Y2',
        ('Y2','a'):'R',('Y2','b'):'Y3',
        ('Y3','a'):'Y3',('Y3','b'):'R',
        ('R','a'):'R',('R','b'):'R'
    },
    'start':'Y1',
    'accept':['Y3']
}

# a sample test

tests = [
    ('aabb',True),('aabb',True),
    ('',False),('a',False),('b',False),
    ('bbbaa',False),('aabbb',False)
]

# want verbose
verbose = True
test_machine(IntersectionDFA(X,Y).construct(),tests)

# further testing ...



running tests ...
Machine Description: 
	states: 16
		 ('X1', 'Y1')
		 ('X1', 'Y2')
		 ('X1', 'Y3')
		 ('X1', 'R')
		 ('X2', 'Y1')
		 ('X2', 'Y2')
		 ('X2', 'Y3')
		 ('X2', 'R')
		 ('X3', 'Y1')
		 ('X3', 'Y2')
		 ('X3', 'Y3')
		 ('X3', 'R')
		 ('R', 'Y1')
		 ('R', 'Y2')
		 ('R', 'Y3')
		 ('R', 'R')
	transitions: 32
		(('X1', 'Y1'), 'b')  ->  ('X1', 'Y2')
		(('X1', 'Y2'), 'b')  ->  ('X1', 'Y3')
		(('X1', 'Y3'), 'b')  ->  ('X1', 'R')
		(('X1', 'R'), 'b')  ->  ('X1', 'R')
		(('X1', 'Y1'), 'a')  ->  ('X2', 'Y1')
		(('X1', 'Y2'), 'a')  ->  ('X2', 'R')
		(('X1', 'Y3'), 'a')  ->  ('X2', 'Y3')
		(('X1', 'R'), 'a')  ->  ('X2', 'R')
		(('X2', 'Y1'), 'b')  ->  ('R', 'Y2')
		(('X2', 'Y2'), 'b')  ->  ('R', 'Y3')
		(('X2', 'Y3'), 'b')  ->  ('R', 'R')
		(('X2', 'R'), 'b')  ->  ('R', 'R')
		(('X2', 'Y1'), 'a')  ->  ('X3', 'Y1')
		(('X2', 'Y2'), 'a')  ->  ('X3', 'R')
		(('X2', 'Y3'), 'a')  ->  ('X3', 'Y3')
		(('X2', 'R'), 'a')  ->  ('X3', 'R')
		(('X3', 'Y1'), 'b')  ->  ('X3', 'Y2')
		(('X3', 'Y2'), 'b

True

### Basic Tests

In [5]:

def basic_test_intersection(dfa1_l, dfa2_l, test_cases_l):
    
    assert len(dfa1_l)==len(test_cases_l)
    assert len(dfa2_l)==len(test_cases_l)

    correct = 0
    num_tests = len(test_cases_l)
    print(f"\n*** Running Basic Tests on {num_tests} machines")
    for i in range(num_tests):
        print("\nExercise",i)
        dfa = IntersectionDFA(dfa1_l[i],dfa2_l[i]).construct()
        if test_machine(dfa,test_cases_l[i],name="machine "+str(i)):
            correct += 1
    print("\n*** correct:",correct,"out of",num_tests)
    if correct==num_tests:
        print("*** passsed")
    else:
        print("*** failed")



In [6]:
basic_test_intersection([X],[Y],[tests])


*** Running Basic Tests on 1 machines

Exercise 0
running tests ...
Machine Description: machine 0
	states: 16
		 ('X1', 'Y1')
		 ('X1', 'Y2')
		 ('X1', 'Y3')
		 ('X1', 'R')
		 ('X2', 'Y1')
		 ('X2', 'Y2')
		 ('X2', 'Y3')
		 ('X2', 'R')
		 ('X3', 'Y1')
		 ('X3', 'Y2')
		 ('X3', 'Y3')
		 ('X3', 'R')
		 ('R', 'Y1')
		 ('R', 'Y2')
		 ('R', 'Y3')
		 ('R', 'R')
	transitions: 32
		(('X1', 'Y1'), 'b')  ->  ('X1', 'Y2')
		(('X1', 'Y2'), 'b')  ->  ('X1', 'Y3')
		(('X1', 'Y3'), 'b')  ->  ('X1', 'R')
		(('X1', 'R'), 'b')  ->  ('X1', 'R')
		(('X1', 'Y1'), 'a')  ->  ('X2', 'Y1')
		(('X1', 'Y2'), 'a')  ->  ('X2', 'R')
		(('X1', 'Y3'), 'a')  ->  ('X2', 'Y3')
		(('X1', 'R'), 'a')  ->  ('X2', 'R')
		(('X2', 'Y1'), 'b')  ->  ('R', 'Y2')
		(('X2', 'Y2'), 'b')  ->  ('R', 'Y3')
		(('X2', 'Y3'), 'b')  ->  ('R', 'R')
		(('X2', 'R'), 'b')  ->  ('R', 'R')
		(('X2', 'Y1'), 'a')  ->  ('X3', 'Y1')
		(('X2', 'Y2'), 'a')  ->  ('X3', 'R')
		(('X2', 'Y3'), 'a')  ->  ('X3', 'Y3')
		(('X2', 'R'), 'a')  ->  ('X3', 'R')